# Load PointClouds in Spark using PDAL
We calculate the Normalized Height for each point.

In [1]:
import geotrellis.pointcloud.spark.Extent3D
import geotrellis.pointcloud.spark.io.hadoop.HadoopPointCloudRDD
import geotrellis.pointcloud.spark.io.s3.S3PointCloudRDD
import geotrellis.pointcloud.spark.tiling.CutPointCloud
import geotrellis.raster.GridExtent
import geotrellis.spark.SpatialKey
import geotrellis.spark.io.hadoop.HadoopCollectionLayerReader
import geotrellis.spark.tiling.LayoutDefinition
import geotrellis.vector.Extent
import io.pdal.Pipeline
import io.pdal.pipeline.{HagFilter, EigenValuesFilter, LasRead, LasWrite, PythonFilter, Read}
import org.apache.hadoop.fs.Path
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}
import spire.syntax.cfor._
import scalaz.stream.Process

In [2]:
implicit val sparkContext = sc

Waiting for a Spark session to start...

sparkContext = org.apache.spark.SparkContext@9eca6f5


Spark app-20180521191653-0000: Some(http://145.100.58.104.surf-hosted.nl:4040)

## Data data paths

In [3]:
val tiles_path = "/data/local/home/ecolidar/"
val tile = "C_25EZ2"
val laz_filepath = tiles_path + tile + ".laz"

tiles_path = /data/local/home/ecolidar/
tile = C_25EZ2
laz_filepath = /data/local/home/ecolidar/C_25EZ2.laz


/data/local/home/ecolidar/C_25EZ2.laz

### Read local and save the result back to a file

In [22]:
//val las_pipelineExpr = LasRead(laz_filepath) ~ HagFilter() ~ FerryFilter(dimensions = "Z=HeightAboveGround") ~ LasWrite(tiles_path + tile + "_hag.laz")
val las_pipelineExpr = LasRead(laz_filepath) ~ HagFilter() ~ LasWrite(tiles_path + tile + "_hag.laz")
val las_pipeline: Pipeline = las_pipelineExpr.toPipeline
las_pipeline.execute()

las_pipelineExpr = List(LasRead(/data/local/home/ecolidar/C_25EZ2.laz,None,None,None,None,None,readers.las), HagFilter(filters.hag), LasWrite(/data/local/home/ecolidar/C_25EZ2_hag.laz,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,writers.las))
las_pipeline = io.pdal.Pipeline@155c10af


io.pdal.Pipeline@155c10af

### Verify if the saved file has HeightAboveGround

### Read HDFS and apply HagFilter

In [55]:
val laz_path = new Path("/user/hadoop/ahn3/C_25EZ2.las")
val pipelineExpr = Read("local") ~ HagFilter() //~ FerryFilter(dimensions = "HeightAboveGround=Z")
val rdd_laz = HadoopPointCloudRDD(laz_path, options = HadoopPointCloudRDD.Options(pipeline = pipelineExpr))

laz_path = /user/hadoop/ahn3/C_25EZ2.las
pipelineExpr = List(Read(local,None,None,None), HagFilter(filters.hag))
rdd_laz = NewHadoopRDD[56] at newAPIHadoopRDD at HadoopPointCloudRDD.scala:76


NewHadoopRDD[56] at newAPIHadoopRDD at HadoopPointCloudRDD.scala:76

### Verify if HeightAboveGround shows up in the schema

In [48]:
rdd_laz.cache()
rdd_laz.map{ case (h, i) => h.schema}.take(1)

Array("{
  "schema":
  {
    "dimensions":
    [
      {
        "name": "X",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Y",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Z",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Intensity",
        "size": 2,
        "type": "unsigned"
      },
      {
        "name": "ReturnNumber",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "NumberOfReturns",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "ScanDirectionFlag",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "EdgeOfFlightLine",
        "size": 1,
        "type": "unsigne...


[{
  "schema":
  {
    "dimensions":
    [
      {
        "name": "X",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Y",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Z",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Intensity",
        "size": 2,
        "type": "unsigned"
      },
      {
        "name": "ReturnNumber",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "NumberOfReturns",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "ScanDirectionFlag",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "EdgeOfFlightLine",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "Classification",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "ScanAngleRank",
        "size": 4,
        "type": "floating"
      },
      {
        "name": "Use

In [49]:
rdd_laz.count()

1

In [15]:
rdd_laz.unpersist()

NewHadoopRDD[0] at newAPIHadoopRDD at HadoopPointCloudRDD.scala:76

In [21]:
rdd_laz.repartition(10)

MapPartitionsRDD[18] at repartition at <console>:45

In [22]:
rdd_laz.flatMap(_._2).getNumPartitions

1

In [30]:
pointCloudTiled.count()

[Stage 9:=====================================================>(995 + 5) / 1000]

45213

In [33]:
pointCloudTiled.getNumPartitions

1000

### Load all the points into RDD and compare Z with NormalizedHeight

In [65]:
val points :RDD[(Double, Double, Double, Byte, Double)] = rdd_laz.flatMap(_._2).mapPartitions{ _.map { packedPoints =>
    var res = new Array[(Double, Double, Double, Byte, Double)](packedPoints.length)
    cfor(0)(_ < packedPoints.length, _ + 1) { i =>
        res(i) = (packedPoints.getX(i), packedPoints.getY(i), packedPoints.getZ(i), packedPoints.getByte(i, dim = "Classification"), packedPoints.getDouble(i, dim = "HeightAboveGround"))
    }
    res
}}.flatMap( m => m).filter(_._4 != 2)//.filter(_._3 > 12)

points = MapPartitionsRDD[82] at filter at <console>:95


MapPartitionsRDD[82] at filter at <console>:95

In [62]:
points.count()

15785689

In [79]:
points.sortBy{ case(x, y, z, c, zh) => (x,y)}.take(10)

[(125000.0,487503.61100000003,10.738,6,7.497999999999999), (125000.0,487512.52400000003,10.867,6,7.644000000000001), (125000.0,487522.228,10.901,6,9.167), (125000.0,488181.435,-0.47500000000000003,9,-0.8190000000000001), (125000.0,488317.335,-0.466,9,-0.020000000000000018), (125000.0,488351.775,-0.47400000000000003,9,-0.028000000000000025), (125000.0,488396.354,-0.47900000000000004,9,-0.03300000000000003), (125000.0,488475.981,-0.481,9,-0.08999999999999997), (125000.0,488810.282,8.955,1,8.016), (125000.0,488861.316,4.473,1,3.635)]

## Tile the pointCloud

In [58]:
val extent = Extent(0, 0, 10, 5)
val gridExtent = GridExtent(extent, 1, 1)  // 10×5 pixels

//val layoutDefinition = LayoutDefinition(extent, TileLayout(layoutCols = 5, layoutRows = 10, tileCols = 10, tileRows = 10))
val layoutDefinition = LayoutDefinition(gridExtent, 10, 5)
val pointCloud = rdd_laz.flatMap(_._2)
val pointCloudTiled = CutPointCloud(pointCloud, layoutDefinition)

extent = Extent(0.0, 0.0, 10.0, 5.0)
gridExtent = GridExtent(Extent(0.0, 0.0, 10.0, 5.0),1.0,1.0)
layoutDefinition = GridExtent(Extent(0.0, 0.0, 10.0, 5.0),1.0,1.0)
pointCloud = MapPartitionsRDD[62] at flatMap at <console>:103
pointCloudTiled = ContextRDD[65] at RDD at ContextRDD.scala:35


ContextRDD[65] at RDD at ContextRDD.scala:35

In [67]:
val points2 :RDD[(SpatialKey, Array[(Double, Double, Double, Byte, Double)])] = pointCloudTiled.mapPartitions{ _.map { case (s, packedPoints) => (s,{
  var res = new Array[(Double, Double, Double, Byte, Double)](packedPoints.length)
  cfor(0)(_ < packedPoints.length, _ + 1) { i =>
    res(i) = (packedPoints.getX(i), packedPoints.getY(i), packedPoints.getZ(i), packedPoints.getByte(i, dim = "Classification"), packedPoints.getDouble(i, dim = "HeightAboveGround"))
  }
  res})
}}
val pointsb = points2.sortByKey().flatMap(m => m._2).filter(_._4 != 2)

[Stage 41:====================================================>(993 + 7) / 1000]

points2 = MapPartitionsRDD[83] at mapPartitions at <console>:97
pointsb = MapPartitionsRDD[88] at filter at <console>:104


MapPartitionsRDD[88] at filter at <console>:104

In [60]:
pointsb.count()

[Stage 32:====================================================>(995 + 5) / 1000]

15785689

In [80]:
pointsb.sortBy{ case(x, y, z, c, zh) => (x,y)}.take(10)

[Stage 81:====================================================>(996 + 4) / 1000]

[(125000.0,487503.61100000003,10.738,6,7.497999999999999), (125000.0,487512.52400000003,10.867,6,7.644000000000001), (125000.0,487522.228,10.901,6,9.167), (125000.0,488181.435,-0.47500000000000003,9,-0.8190000000000001), (125000.0,488317.335,-0.466,9,-0.020000000000000018), (125000.0,488351.775,-0.47400000000000003,9,-0.028000000000000025), (125000.0,488396.354,-0.47900000000000004,9,-0.03300000000000003), (125000.0,488475.981,-0.481,9,-0.08999999999999997), (125000.0,488810.282,8.955,1,8.016), (125000.0,488861.316,4.473,1,3.635)]

### Verify if the created file and uploaded to HDFS has Normalized Height

In [12]:
val laz_path = new Path("/user/hadoop/ahn3/C_25EZ2_hag.laz")
val rdd_laz = HadoopPointCloudRDD(laz_path)
rdd_laz.cache()
rdd_laz.count()

laz_path = /user/hadoop/ahn3/C_25EZ2_hag.laz
rdd_laz = NewHadoopRDD[0] at newAPIHadoopRDD at HadoopPointCloudRDD.scala:76


1

In [42]:
val points :RDD[(Double, Double, Double, Byte)] = rdd_laz.flatMap(_._2).mapPartitions{ _.map { packedPoints =>
  var res = new Array[(Double, Double, Double, Byte)](packedPoints.length)
  cfor(0)(_ < packedPoints.length, _ + 1) { i =>
    res(i) = (packedPoints.getX(i), packedPoints.getY(i), packedPoints.getZ(i), packedPoints.getByte(i, dim = "Classification"))
  }
  res
}}.flatMap( m => m).filter(_._4 != 2).filter(_._3 > 0)
val points10 = points.take(10)

points = MapPartitionsRDD[32] at filter at <console>:95
points10 = Array()


[]

# Read from S3

In [8]:
val output = sys. Process("env",
                     None,
                     "AWS_ACCESS_KEY_ID" -> "A24H1RIGV4RKFGXJTEMS",
                     "AWS_SECRET_ACCESS_KEY" -> "5jd7ARCOi/XVjLzXqT5wA1NSgjmUo9mYJBgyGyIh")


output = Emit(WrappedArray(env, None, (AWS_ACCESS_KEY_ID,A24H1RIGV4RKFGXJTEMS), (AWS_SECRET_ACCESS_KEY,5jd7ARCOi/XVjLzXqT5wA1NSgjmUo9mYJBgyGyIh)))


Emit(WrappedArray(env, None, (AWS_ACCESS_KEY_ID,A24H1RIGV4RKFGXJTEMS), (AWS_SECRET_ACCESS_KEY,5jd7ARCOi/XVjLzXqT5wA1NSgjmUo9mYJBgyGyIh)))

In [5]:
val s3_laz_path = "ahn3"
val s3_files = "C_25EZ2.laz"
val s3_file_out = "C_25EZ2_hag.laz"

val pipelineExpr = Read("local") ~ HagFilter()//~ LasWrite(s3_laz_path + "C_25EZ2_hag.laz")

val s3_rdd_laz = S3PointCloudRDD(s3_laz_path, s3_files, options = S3PointCloudRDD.Options(pipeline = pipelineExpr))

s3_laz_path = ahn3
s3_files = C_25EZ2.laz
s3_file_out = C_25EZ2_hag.laz
pipelineExpr = List(Read(local,None,None,None), HagFilter(filters.hag))
s3_rdd_laz = NewHadoopRDD[1] at newAPIHadoopRDD at S3PointCloudRDD.scala:88


lastException: Throwable = null


NewHadoopRDD[1] at newAPIHadoopRDD at S3PointCloudRDD.scala:88

In [6]:
s3_rdd_laz.count()

Name: com.amazonaws.services.s3.model.AmazonS3Exception
Message: The AWS Access Key Id you provided does not exist in our records. (Service: Amazon S3; Status Code: 403; Error Code: InvalidAccessKeyId; Request ID: 4D69CC537F251B7B; S3 Extended Request ID: bWyin9n4shEdxKOTQXM9mc2CyJHVRYe7cTMGCgms+hhJnElVFVcNoN9KaddQ9U17/LXkRa1Hoio=)
StackTrace:   at com.amazonaws.http.AmazonHttpClient$RequestExecutor.handleErrorResponse(AmazonHttpClient.java:1632)
  at com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeOneRequest(AmazonHttpClient.java:1304)
  at com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeHelper(AmazonHttpClient.java:1058)
  at com.amazonaws.http.AmazonHttpClient$RequestExecutor.doExecute(AmazonHttpClient.java:743)
  at com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeWithTimer(AmazonHttpClient.java:717)
  at com.amazonaws.http.AmazonHttpClient$RequestExecutor.execute(AmazonHttpClient.java:699)
  at com.amazonaws.http.AmazonHttpClient$RequestExecutor.acc

In [8]:
val extent = Extent(0, 0, 10, 5)
val gridExtent = GridExtent(extent, 1, 1)  // 10×5 pixels

//val layoutDefinition = LayoutDefinition(extent, TileLayout(layoutCols = 5, layoutRows = 10, tileCols = 10, tileRows = 10))
val layoutDefinition = LayoutDefinition(gridExtent, 10, 5)
val pointCloud = s3_rdd_laz.flatMap(_._2)
val pointCloudTiled = CutPointCloud(pointCloud, layoutDefinition)

extent = Extent(0.0, 0.0, 10.0, 5.0)
gridExtent = GridExtent(Extent(0.0, 0.0, 10.0, 5.0),1.0,1.0)
layoutDefinition = GridExtent(Extent(0.0, 0.0, 10.0, 5.0),1.0,1.0)
pointCloud = MapPartitionsRDD[1] at flatMap at <console>:71
pointCloudTiled = ContextRDD[4] at RDD at ContextRDD.scala:35


ContextRDD[4] at RDD at ContextRDD.scala:35

In [4]:
val sonnets = sc.textFile("s3a://files/sonnets.txt")
val counts = sonnets.flatMap(line => line.split(" ")).map(word => (word, 1)).reduceByKey(_ + _)
//counts.saveAsTextFile("s3a://files/output")

Waiting for a Spark session to start...

sonnets = s3a://files/sonnets.txt MapPartitionsRDD[1] at textFile at <console>:47
counts = ShuffledRDD[4] at reduceByKey at <console>:48


ShuffledRDD[4] at reduceByKey at <console>:48

In [5]:
sonnets.count()

Name: java.lang.IllegalAccessError
Message: tried to access method com.google.common.base.Stopwatch.<init>()V from class org.apache.hadoop.mapred.FileInputFormat
StackTrace:   at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:312)
  at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:200)
  at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
  at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
  at scala.Option.getOrElse(Option.scala:121)
  at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
  at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
  at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
  at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
  at scala.Option.getOrElse(Option.scala:121)
  at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
  at org.apache.spark.SparkContext.runJob(SparkContext.scala:2092)
  at org.apache.spark.rdd.